Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 72.1 MB/s eta 0:00:00


QUANTUM COUNTING ALGORİTMASI

Cirq ile Quantum Counting Örneği

In [3]:
import cirq
import numpy as np

# 2 qubitlik arama register + 2 qubit sayaç register
search_qubits = [cirq.GridQubit(0,i) for i in range(2)]
count_qubits = [cirq.GridQubit(1,i) for i in range(2)]

# Basit oracle (çözüm |11>)
oracle = cirq.Circuit(cirq.Z(search_qubits[0]).controlled_by(search_qubits[1]))

# Grover diffusion operatörü
def diffusion(qs):
    return [cirq.H.on_each(*qs),
            cirq.X.on_each(*qs),
            cirq.Z.on_each(*qs),
            cirq.X.on_each(*qs),
            cirq.H.on_each(*qs)]

# Sayaç register için Hadamard
circuit = cirq.Circuit()
circuit.append(cirq.H.on_each(*count_qubits))
circuit.append(cirq.H.on_each(*search_qubits))

# Basit 1 Grover iterasyonu
circuit.append(oracle)
circuit.append(diffusion(search_qubits))

# Ölçüm
circuit.append(cirq.measure(*search_qubits, key='result'))
print(circuit)


(0, 0): ───H───@───H───X───Z───X───H───M('result')───
               │                       │
(0, 1): ───H───@───H───X───Z───X───H───M─────────────

(1, 0): ───H─────────────────────────────────────────

(1, 1): ───H─────────────────────────────────────────


Qiskit ile Quantum Counting Örneği

In [4]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import GroverOperator
import numpy as np

# 2 qubitlik arama register + 2 qubit sayaç
qc = QuantumCircuit(4,2)

# Sayaç register Hadamard
qc.h([0,1])

# Arama register başlangıç (Hadamard)
qc.h([2,3])

# Basit oracle: çözüm |11>
qc.cz(2,3)

# Grover diffusion
qc.h([2,3])
qc.x([2,3])
qc.cz(2,3)
qc.x([2,3])
qc.h([2,3])

# Ölçüm
qc.measure([2,3],[0,1])

backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=1024).result()
counts = result.get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Quantum Counting Simülasyonu

In [5]:
import qutip as qt
import numpy as np

# 2 qubit arama register
n = 2
psi0 = qt.tensor([qt.basis(2,0) for _ in range(n)])
# Grover operatörü (basit oracle + diffusion)
oracle = qt.tensor([qt.qeye(2), qt.qeye(2)])
diffusion = qt.tensor([qt.qeye(2), qt.qeye(2)])
G = oracle * diffusion

# Quantum counting: QPE simülasyonu (basitleştirilmiş)
theta = np.pi / 3  # varsayım: çözüm sayısını kodlayan açı
M = 2  # tahmini çözüm sayısı
print("Tahmini çözüm sayısı M =", M)


Tahmini çözüm sayısı M = 2


AMPLITUDE ESTIMATION ALGORİTMASI

Qiskit ile Küçük Örnek

In [7]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import GroverOperator
import numpy as np

# Örnek: a = 0.25 olacak şekilde A operatörü
qc = QuantumCircuit(2,1)

# Başlangıç devresi: sqrt(a)|1> + sqrt(1-a)|0>
theta = 2*np.arcsin(np.sqrt(0.25))
qc.ry(theta, 1)
qc.cx(1, 0)  # Basit Grover benzeri kontrol

# Ölçüm
qc.measure(1,0)

backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=1024).result()
counts = result.get_counts()
a_estimate = counts.get('1',0)/1024
print(f"Tahmini olasılık a ≈ {a_estimate}")


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

Cirq ile Küçük Örnek

In [8]:
import cirq
import numpy as np

q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit()

# A operatörü: a=0.25
theta = 2 * np.arcsin(np.sqrt(0.25))
circuit.append(cirq.ry(theta)(q1))
circuit.append(cirq.CNOT(q1, q0))

# Ölçüm
circuit.append(cirq.measure(q1, key='m'))

simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=1024)
a_estimate = np.mean(result.measurements['m'])
print(f"Tahmini olasılık a ≈ {a_estimate}")


Tahmini olasılık a ≈ 0.24609375


Qutip


In [9]:
import numpy as np
from qutip import basis, tensor, qeye, rx, cnot, measurement_statistics_povm

# Başlangıç durumu |0>|0>
psi0 = tensor(basis(2,0), basis(2,0))

# Hedef olasılık a = 0.25
a = 0.25
theta = 2 * np.arcsin(np.sqrt(a))

# A operatörü: |0>|0> -> sqrt(1-a)|ψ0>|0> + sqrt(a)|ψ1>|1>
# Burada 2. qubit üzerinde rotasyon
U_A = rx(theta)
psi = U_A * psi0.ptrace(1)  # sadece hedef qubit

# Toplam durum (basitleştirilmiş)
psi_total = tensor(basis(2,0), psi)

# POVM ölçümü
p0 = (psi_total.dag() * tensor(basis(2,0)*basis(2,0).dag(), qeye(2)) * psi_total).tr()
p1 = (psi_total.dag() * tensor(basis(2,1)*basis(2,1).dag(), qeye(2)) * psi_total).tr()

print(f"Tahmini olasılık a ≈ {p1:.3f}")


ImportError: cannot import name 'rx' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)

QUANTUM RANDOM WALK ALGORİTMALARI

Cirq ile 1D Kuantum Random Walk

In [11]:
import cirq
import numpy as np

# Qubitler: 1 coin, 3 konum (2^2=4 konum)
coin = cirq.NamedQubit("coin")
positions = [cirq.NamedQubit(f"pos{i}") for i in range(2)]  # 2 qubit = 4 konum

# Coin operatörü (Hadamard)
def coin_op():
    return cirq.H(coin)

# Shift operatörü
def shift_op():
    # Basit örnek: 1 adım sola veya sağa
    return [
        cirq.CNOT(coin, positions[0]),
        cirq.CNOT(coin, positions[1])
    ]

# Devre oluşturma
circuit = cirq.Circuit()
circuit.append(coin_op())
circuit.append(shift_op())

# Birkaç adım tekrarlamak için
steps = 3
for _ in range(steps):
    circuit.append(coin_op())
    circuit.append(shift_op())

print(circuit)

# Simülasyon
simulator = cirq.Simulator()
result = simulator.simulate(circuit)
print(result.final_state_vector)


coin: ───H───@───@───H───@───@───H───@───@───H───@───@───
             │   │       │   │       │   │       │   │
pos0: ───────X───┼───────X───┼───────X───┼───────X───┼───
                 │           │           │           │
pos1: ───────────X───────────X───────────X───────────X───
[ 0.+0.j  0.+0.j  0.+0.j  1.+0.j -0.+0.j -0.+0.j -0.+0.j  0.+0.j]


Qiskit ile 1D Kuantum Random Walk

In [12]:
from qiskit import QuantumCircuit, Aer, execute
import numpy as np

# Qubitler: 1 coin, 2 konum qubit
qc = QuantumCircuit(3)

# Başlangıç durumu: coin |0>, konum |00>
qc.h(0)  # Coin Hadamard

# Shift operatörü (basitleştirilmiş)
qc.cx(0,1)
qc.cx(0,2)

# Birkaç adım
steps = 3
for _ in range(steps):
    qc.h(0)      # coin
    qc.cx(0,1)   # shift
    qc.cx(0,2)

print(qc.draw())

# Simülasyon
backend = Aer.get_backend('statevector_simulator')
result = execute(qc, backend).result()
state = result.get_statevector()
print(state)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile 1D Kuantum Random Walk (state-vector tabanlı)

In [13]:
import numpy as np
from qutip import basis, tensor, qeye, hadamard_transform

# 1 coin qubit + 2 konum qubit = 4 konum
coin_dim = 2
pos_dim = 4

# Başlangıç: coin |0>, konum |0>
psi = tensor(basis(2,0), basis(4,0))

# Coin operatörü (Hadamard)
H_coin = hadamard_transform(1)

# Shift operatörü: basit mod 4 adımı
S = np.zeros((8,8))
for i in range(4):
    S[i*2, i*2] = 1  # coin=0, sola
    S[i*2+1, (i*2+3)%8] = 1  # coin=1, sağa

# 1 adım operatörü
U = S @ np.kron(H_coin.full(), np.eye(4))

# 3 adım QRW
for _ in range(3):
    psi = U @ psi.full()

print("Final state vector:")
print(psi)


ImportError: cannot import name 'hadamard_transform' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)

QUANTUM GIBBS SAMPLING

QuTiP ile Quantum Gibbs State

In [14]:
import numpy as np
from qutip import thermal_dm, basis, tensor, sigmax, sigmaz, qeye, Qobj, expect

# Örnek: 1 qubit Hamiltonyen
beta = 1.0  # ters sıcaklık
H = 0.5 * sigmaz()  # simple Hamiltonian

# Gibbs durumu (thermal state)
rho_beta = thermal_dm(H.dims[0][0], beta * 1.0)  # qubit için

print("Gibbs density matrix:")
print(rho_beta)

# Örnek: ölçüm (sigma_z beklenen değeri)
exp_val = expect(sigmaz(), rho_beta)
print("Expectation <Z>:", exp_val)


Gibbs density matrix:
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dia, isherm=True
Qobj data =
[[0.66666667 0.        ]
 [0.         0.33333333]]
Expectation <Z>: 0.3333333333333333


Cirq ile Imaginary Time Evolution (yaklaşık QITE)

In [15]:
import cirq
import numpy as np

# 1 qubit
q = cirq.GridQubit(0,0)
circuit = cirq.Circuit()

# Basit Hamiltonian: H = Z
H = cirq.Z(q)

# Başlangıç durumu: |0>
circuit.append(cirq.H(q))  # süperpozisyon

# Imaginary time evolution: U = exp(-beta H)
beta = 1.0
theta = -1j * beta  # -i H t, t -> -i beta
circuit.append(cirq.rz(2 * beta)(q))  # yaklaşık küçük beta için

simulator = cirq.Simulator()
result = simulator.simulate(circuit)
print("State vector after imaginary time evolution:")
print(result.final_state_vector)


State vector after imaginary time evolution:
[0.38205147-0.59500986j 0.3820514 +0.59500986j]


Qiskit ile Termal State Hazırlama (basit örnek)

In [16]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.quantum_info import DensityMatrix, partial_trace
import numpy as np

# Basit Hamiltonian: H = Z
beta = 1.0
p0 = np.exp(-beta * 0.5)
p1 = np.exp(beta * 0.5)
Z = p0 + p1
p0 /= Z
p1 /= Z

# Density matrix: rho = diag(p0, p1)
rho = DensityMatrix([[p0,0],[0,p1]])
print("Gibbs state density matrix:")
print(rho.data)

# Qiskit simülasyonu ile ölçüm
backend = Aer.get_backend('statevector_simulator')
qc = QuantumCircuit(1)
result = execute(qc, backend, initial_state=rho).result()
state = result.get_statevector()
print("Statevector (from Gibbs state):", state)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QUANTUM MONTE CARLO

QuTiP ile Variational Monte Carlo (VMC) örneği

In [17]:
import numpy as np
from qutip import sigmax, sigmaz, basis, expect

# 1-qubit Hamiltonian H = X + Z
H = sigmax() + sigmaz()

# Trial state |psi_T> = cos(theta)|0> + sin(theta)|1>
theta = np.pi/4
psi_T = np.cos(theta)*basis(2,0) + np.sin(theta)*basis(2,1)

# Energy expectation
E = expect(H, psi_T)
print("Trial state energy <H>:", E)


Trial state energy <H>: 1.0


Cirq ile Quantum Monte Carlo (Amplitude Estimation tabanlı)

In [18]:
import cirq
import numpy as np

# 1-qubit örnek
q = cirq.GridQubit(0, 0)
circuit = cirq.Circuit()

# Başlangıç: |0> -> |psi>
circuit.append(cirq.H(q))

# Energy measurement operator: Z
# Örnek: <H> = <Z>
simulator = cirq.Simulator()
result = simulator.simulate(circuit)
state = result.final_state_vector

# Energy expectation <Z>
E = abs(state[0])**2 - abs(state[1])**2
print("Energy expectation <Z>:", E)


Energy expectation <Z>: 0.0


Qiskit ile Quantum Monte Carlo / Amplitude Estimation

In [19]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.quantum_info import Statevector
import numpy as np

# Trial state |psi> = H|0> = (|0> + |1>)/√2
qc = QuantumCircuit(1)
qc.h(0)

backend = Aer.get_backend('statevector_simulator')
result = execute(qc, backend).result()
state = Statevector(result.get_statevector())

# Energy operator H = Z
# <Z> = |0|^2 - |1|^2
E = np.abs(state.data[0])**2 - np.abs(state.data[1])**2
print("Energy expectation <Z>:", E)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)